In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Step 1: Load and preprocess the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)


In [2]:

# Step 2: Define the model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)
        return x

model = MLP()

# Step 3: Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

import time

# 开始计时
start_time = time.time()



# Step 4: Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    if epoch%10==0 or epoch==num_epochs - 1:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader.dataset):.4f}')

# 结束计时
end_time = time.time()

# 计算时间差
elapsed_time = end_time - start_time

print(f"训练时间：{elapsed_time:.2f}秒")

# Step 5: Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the test set: {(100 * correct / total):.2f}%')




Epoch [1/100], Loss: 2.2972
Epoch [11/100], Loss: 1.6696
Epoch [21/100], Loss: 1.6351
Epoch [31/100], Loss: 1.6255
Epoch [41/100], Loss: 1.6193
Epoch [51/100], Loss: 1.6147
Epoch [61/100], Loss: 1.6108
Epoch [71/100], Loss: 1.6073
Epoch [81/100], Loss: 1.6041
Epoch [91/100], Loss: 1.6010
Epoch [100/100], Loss: 1.5988
训练时间：284.93秒
Accuracy on the test set: 85.96%


In [3]:
with open('../dataset/mnist', 'w') as file:
    for images, labels in train_loader:
        for i in range(len(images)):
            ssub = images[i][0].reshape(1, -1).tolist()[0]
            for j in ssub:
                print(j, end=' ', file=file)

            l = int(labels[i].tolist())
            temp = [0 for i in range(10)]
            temp[l] = 1
            for j in temp:
                print(j, end=' ', file=file)

            # break
        # break
 

In [4]:
with open('../dataset/mnist_test', 'w') as file:
    for images, labels in test_loader:
        for i in range(len(images)):
            ssub = images[i][0].reshape(1, -1).tolist()[0]
            for j in ssub:
                print(j, end=' ', file=file)

            l = int(labels[i].tolist())
            temp = [0 for i in range(10)]
            temp[l] = 1
            for j in temp:
                print(j, end=' ', file=file)

            # break
        # break
